# Natural Language Processing Lab

In this lab we will further explore Scikit's and NLTK's capabilities to process text. We will use the 20 Newsgroup dataset, which is provided by Scikit-Learn.

In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [2]:
from sklearn.datasets import fetch_20newsgroups

In [3]:

categories = [
    'alt.atheism',
    'talk.religion.misc',
    'comp.graphics',
    'sci.space',
]

data_train = fetch_20newsgroups(subset='train', categories=categories,
                                shuffle=True, random_state=42,
                                remove=('headers', 'footers', 'quotes'))

data_test = fetch_20newsgroups(subset='test', categories=categories,
                               shuffle=True, random_state=42,
                               remove=('headers', 'footers', 'quotes'))

No handlers could be found for logger "sklearn.datasets.twenty_newsgroups"


## 1. Data inspection

We have downloaded a few newsgroup categories and removed headers, footers and quotes.

Let's inspect them.

1. What data taype is `data_train`
> sklearn.datasets.base.Bunch
- Is it like a list? Or like a Dictionary? or what?
> Dict
- How many data points does it contain?
- Inspect the first data point, what does it look like?
> A blurb of text

In [11]:
print data_train.data[:5]

[u"Hi,\n\nI've noticed that if you only save a model (with all your mapping planes\npositioned carefully) to a .3DS file that when you reload it after restarting\n3DS, they are given a default position and orientation.  But if you save\nto a .PRJ file their positions/orientation are preserved.  Does anyone\nknow why this information is not stored in the .3DS file?  Nothing is\nexplicitly said in the manual about saving texture rules in the .PRJ file. \nI'd like to be able to read the texture rule information, does anyone have \nthe format for the .PRJ file?\n\nIs the .CEL file format available from somewhere?\n\nRych", u'\n\nSeems to be, barring evidence to the contrary, that Koresh was simply\nanother deranged fanatic who thought it neccessary to take a whole bunch of\nfolks with him, children and all, to satisfy his delusional mania. Jim\nJones, circa 1993.\n\n\nNope - fruitcakes like Koresh have been demonstrating such evil corruption\nfor centuries.', u"\n >In article <1993Apr19.02

## 2. Bag of Words model

Let's train a model using a simple count vectorizer

1. Initialize a standard CountVectorizer and fit the training data
- how big is the feature dictionary
- repeat eliminating english stop words
- is the dictionary smaller?
- transform the training data using the trained vectorizer
- what are the 20 words that are most common in the whole corpus?
- what are the 20 most common words in each of the 4 classes?
- evaluate the performance of a Lotistic Regression on the features extracted by the CountVectorizer
    - you will have to transform the test_set too. Be carefule to use the trained vectorizer, without re-fitting it
- try the following 3 modification:
    - restrict the max_features
    - change max_df and min_df
    - use a fixed vocabulary of size 80 combining the 20 most common words per group found earlier
- for each of the above print a confusion matrix and investigate what gets mixed
> Anwer: not surprisingly if we reduce the feature space we lose accuracy
- print out the number of features for each model

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [50]:
v = CountVectorizer()
x_train = v.fit_transform(data_train.data)

In [51]:
x_train.shape

(2034, 26879)

In [49]:
v = CountVectorizer(stop_words="english")
x_train = v.fit_transform(data_train.data)
x_train.shape

(2034, 26576)

It is smaller by about 300

In [60]:
# what are the 20 words that are most common in the whole corpus?
most20common = CountVectorizer(max_features=20, stop_words='english')
most20common.fit_transform(data_train.data)
most20common.vocabulary_

{u'data': 0,
 u'does': 1,
 u'don': 2,
 u'edu': 3,
 u'god': 4,
 u'good': 5,
 u'graphics': 6,
 u'image': 7,
 u'jesus': 8,
 u'just': 9,
 u'know': 10,
 u'like': 11,
 u'nasa': 12,
 u'people': 13,
 u'say': 14,
 u'space': 15,
 u'think': 16,
 u'time': 17,
 u'use': 18,
 u'way': 19}

In [67]:
data_test.target_names

['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']

In [69]:
# what are the 20 most common words in each of the 4 classes?
X = pd.DataFrame(data_train.data, columns =['data'])
y = pd.DataFrame(data_train.target, columns =['class'])
final = pd.concat([X, y], axis = 1)
final.head()

,data,class
0,"Hi,\n\nI've noticed that if you only save a mo...",1
1,"\n\nSeems to be, barring evidence to the contr...",3
2,\n >In article <1993Apr19.020359.26996@sq.sq.c...,2
3,I have a request for those who would like to s...,0
4,AW&ST had a brief blurb on a Manned Lunar Exp...,2


In [97]:
li = []
for i in range(4):
    mc420 = CountVectorizer(max_features=20, stop_words='english')
    mc420.fit_transform(final[final['class'] == i]['data'])
    print i, [x for x in mc420.vocabulary_]
    li.append([x for x in mc420.vocabulary_])

0 [u'people', u'time', u'argument', u'say', u'religion', u'atheists', u'don', u'jesus', u'does', u'way', u'true', u'atheism', u'said', u'just', u'think', u'bible', u'like', u'god', u'believe', u'know']
1 [u'format', u'data', u'image', u'gif', u'ftp', u'graphics', u'does', u'software', u'available', u'use', u'pub', u'like', u'images', u'file', u'edu', u'jpeg', u'color', u'program', u'files', u'know']
2 [u'people', u'time', u'data', u'just', u'year', u'space', u'launch', u'orbit', u'new', u'don', u'lunar', u'shuttle', u'like', u'earth', u'satellite', u'moon', u'program', u'mission', u'nasa', u'use']
3 [u'life', u'people', u'time', u'say', u'jesus', u'does', u'way', u'think', u'don', u'said', u'just', u'did', u'bible', u'good', u'believe', u'point', u'like', u'god', u'christian', u'know']


In [81]:
# evaluate the performance of a Logistic Regression on the features extracted by the CountVectorizer
# you will have to transform the test_set too. Be carefule to use the trained vectorizer, without re-fitting it
def score(v):
    X = v.fit_transform(data_train.data)
    y = v.transform(data_test.data)

    lr = LogisticRegression()
    lr.fit(X, data_train.target)
    y_pred = lr.predict(y)
    print 'score:', lr.score(y, data_test.target)
    print 'con mat: \n', confusion_matrix(data_test.target, y_pred)
    print 'number of words:', len(v.vocabulary_)
    
v = CountVectorizer(stop_words='english')
score(v)

score: 0.745011086475
con mat: 
[[187  16  46  70]
 [ 13 345  28   3]
 [ 22  23 333  16]
 [ 67  14  27 143]]
number of words: 26576


In [83]:
# try the following 3 modification:
# restrict the max_features

print "significantly worse with limited features"
v = CountVectorizer(stop_words='english', max_features=20)
score(v)

significantly worse with limited features
score: 0.49519586105
con mat: 
[[159  98  30  32]
 [ 30 296  55   8]
 [ 47 147 186  14]
 [112  91  19  29]]
number of words: 20


In [96]:
# change max_df and min_df
v = CountVectorizer(stop_words='english', min_df=0.1,max_df=0.9)
score(v)

score: 0.441241685144
con mat: 
[[116 125  41  37]
 [ 30 287  67   5]
 [ 39 181 170   4]
 [ 88 112  27  24]]
number of words: 18


In [ ]:
# use a fixed vocabulary of size 80 combining the 20 most common words per group found earlier

In [ ]:
v = CountVectorizer(li, stop_words='english')
score(v)

## 3. Hashing and TF-IDF

Let's see if Hashing or TF-IDF improves the accuracy.

1. Initialize a HashingVectorizer and repeat the test with no restriction on the number of features
- does the score improve with respect to the count vectorizer?
    - can you change any of the default parameters to improve it?
- print out the number of features for this model
- Initialize a TF-IDF Vectorizer and repeat the analysis above
- can you improve on your best score above?
    - can you change any of the default parameters to improve it?
- print out the number of features for this model

In [116]:
from sklearn.feature_extraction.text import HashingVectorizer
hv = HashingVectorizer()
X = hv.fit_transform(data_train.data)
y = hv.transform(data_test.data)

lr = LogisticRegression()
lr.fit(X, data_train.target)
y_pred = lr.predict(y)
print 'score:', lr.score(y, data_test.target)
print 'con mat: \n', confusion_matrix(data_test.target, y_pred)
print 'number of features:', v.n_features

score: 0.674057649667
con mat: 
[[180  41  54  44]
 [ 15 337  34   3]
 [ 29  47 315   3]
 [100  29  42  80]]
number of features: 1048576


Score is slightly worse than the CountVectorizer

In [121]:
# Initialize a TF-IDF Vectorizer and repeat the analysis above
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(stop_words='english')
X = tv.fit_transform(data_train.data)
y = tv.transform(data_test.data)

lr = LogisticRegression()
lr.fit(X, data_train.target)
y_pred = lr.predict(y)
print 'score:', lr.score(y, data_test.target)
print 'con mat: \n', confusion_matrix(data_test.target, y_pred)
print 'number of words:', len(tv.vocabulary_)

score: 0.747967479675
con mat: 
[[198  15  65  41]
 [  8 351  29   1]
 [ 17  21 356   0]
 [ 82  16  46 107]]
number of words: 26576


## 4. Classifier comparison

Of all the vectorizers tested above, choose one that has a reasonable performance with a manageable number of features and compare the performance of these models:

- KNN
- Logistic Regression
- Decision Trees
- Support Vector Machine
- Random Forest
- Extra Trees

In order to speed up the calculation it's better to vectorize the data only once and then compare the models.

In [129]:
models = [KNeighborsClassifier(), LogisticRegression(), DecisionTreeClassifier(),
          SVC(), RandomForestClassifier(), ExtraTreesClassifier()]

def tester(model, v):
    X = v.fit_transform(data_train.data)
    y = v.transform(data_test.data)
    print model
    mod = model
    mod.fit(X, data_train.target)
    y_pred = mod.predict(y)
    print 'score:', mod.score(y, data_test.target)
    print  'con mat:','\n', confusion_matrix(data_test.target, y_pred)
    print 'number of words:', len(v.vocabulary_)
v = CountVectorizer(stop_words='english', max_df=0.5)
for i in models:
    tester(i, v)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
score: 0.339246119734
con mat: 
[[199  33  21  66]
 [184 119  20  66]
 [191  45  74  84]
 [148  22  14  67]]
number of words: 26576
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
score: 0.745011086475
con mat: 
[[187  16  46  70]
 [ 13 345  28   3]
 [ 22  23 333  16]
 [ 67  14  27 143]]
number of words: 26576
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='be

## Bonus: Other classifiers

Adapt the code from [this example](http://scikit-learn.org/stable/auto_examples/text/document_classification_20newsgroups.html#example-text-document-classification-20newsgroups-py) to compare across all the classifiers suggested and to display the final plot

## Bonus: NLTK

NLTK is a vast library. Can you find some interesting bits to share with classmates?
Start here: http://www.nltk.org/